In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data


In [2]:
len(documents)

11314

In [3]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [4]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [6]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [8]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [10]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [11]:
print(dictionary[66])

faith


In [12]:
len(dictionary)

64281

In [13]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.013*"would" + 0.012*"like" + 0.010*"know" + 0.009*"anyone"')
(1, '0.027*"president" + 0.012*"jobs" + 0.011*"going" + 0.009*"clinton"')
(2, '0.006*"university" + 0.006*"national" + 0.005*"health" + 0.005*"states"')
(3, '0.043*"space" + 0.016*"nasa" + 0.009*"launch" + 0.009*"earth"')
(4, '0.013*"phillies" + 0.010*"plane" + 0.010*"strip" + 0.010*"kidney"')
(5, '0.014*"navy" + 0.010*"naval" + 0.007*"ciphertext" + 0.007*"bullets"')
(6, '0.008*"used" + 0.006*"system" + 0.006*"number" + 0.006*"power"')
(7, '0.028*"char" + 0.026*"ground" + 0.025*"wire" + 0.014*"wiring"')
(8, '0.022*"jesus" + 0.011*"bible" + 0.011*"christian" + 0.011*"church"')
(9, '0.023*"players" + 0.013*"hockey" + 0.010*"team" + 0.008*"league"')
(10, '0.025*"file" + 0.017*"program" + 0.012*"window" + 0.011*"output"')
(11, '0.016*"available" + 0.015*"mail" + 0.013*"software" + 0.012*"information"')
(12, '0.027*"game" + 0.020*"team" + 0.019*"games" + 0.016*"play"')
(13, '0.024*"government" + 0.012*"encryption" + 0.009*"

In [14]:
print(ldamodel.print_topics())

[(0, '0.013*"would" + 0.012*"like" + 0.010*"know" + 0.009*"anyone" + 0.008*"thanks" + 0.008*"good" + 0.006*"also" + 0.006*"problem" + 0.005*"please" + 0.005*"used"'), (1, '0.027*"president" + 0.012*"jobs" + 0.011*"going" + 0.009*"clinton" + 0.009*"think" + 0.008*"said" + 0.007*"package" + 0.007*"administration" + 0.007*"know" + 0.007*"work"'), (2, '0.006*"university" + 0.006*"national" + 0.005*"health" + 0.005*"states" + 0.005*"april" + 0.005*"research" + 0.004*"american" + 0.004*"years" + 0.004*"medical" + 0.004*"washington"'), (3, '0.043*"space" + 0.016*"nasa" + 0.009*"launch" + 0.009*"earth" + 0.008*"satellite" + 0.007*"shuttle" + 0.007*"orbit" + 0.006*"lunar" + 0.006*"center" + 0.006*"flight"'), (4, '0.013*"phillies" + 0.010*"plane" + 0.010*"strip" + 0.010*"kidney" + 0.008*"cincinnati" + 0.008*"torque" + 0.008*"dram" + 0.007*"copper" + 0.007*"engines" + 0.006*"contacts"'), (5, '0.014*"navy" + 0.010*"naval" + 0.007*"ciphertext" + 0.007*"bullets" + 0.006*"forsale" + 0.006*"knife" + 0

In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [16]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.123443455), (6, 0.022624971), (9, 0.16392995), (16, 0.15564439), (17, 0.37720278), (18, 0.14583711)]
1 번째 문서의 topic 비율은 [(0, 0.041380007), (6, 0.032820933), (11, 0.09015598), (16, 0.39606035), (17, 0.4197981)]
2 번째 문서의 topic 비율은 [(0, 0.050472505), (2, 0.060189977), (6, 0.16185333), (16, 0.25174692), (17, 0.46359065)]
3 번째 문서의 topic 비율은 [(0, 0.20613742), (6, 0.13759997), (11, 0.09899421), (13, 0.28407463), (16, 0.26125592)]
4 번째 문서의 topic 비율은 [(7, 0.24252856), (10, 0.09126397), (12, 0.22788608), (16, 0.40868267)]


In [17]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [18]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,17.0,0.3773,"[(2, 0.12343513), (6, 0.022614332), (9, 0.1639..."
1,1,17.0,0.4198,"[(0, 0.04138591), (6, 0.032820813), (11, 0.090..."
2,2,17.0,0.4636,"[(0, 0.050525572), (2, 0.060187917), (6, 0.161..."
3,3,13.0,0.2841,"[(0, 0.20590673), (6, 0.13765396), (11, 0.0990..."
4,4,16.0,0.4087,"[(7, 0.24252783), (10, 0.09125266), (12, 0.227..."
5,5,8.0,0.3264,"[(5, 0.1385554), (6, 0.17215122), (8, 0.326406..."
6,6,19.0,0.2632,"[(0, 0.2161445), (6, 0.17970574), (8, 0.062680..."
7,7,17.0,0.3930,"[(0, 0.043904487), (2, 0.021551333), (6, 0.120..."
8,8,0.0,0.3698,"[(0, 0.3698364), (1, 0.110374816), (2, 0.19751..."
9,9,0.0,0.5735,"[(0, 0.5734689), (2, 0.01248991), (6, 0.254971..."
